In [2]:
#Import libraries

In [3]:
import torch
import torch.nn as nn
import string

In [4]:
input_ = "Brand mapping is used to understand the positioning of a brand with respect to the attributes in any brand".lower()

In [5]:
input_

'brand mapping is used to understand the positioning of a brand with respect to the attributes in any brand'

In [6]:
seq_len = len(input_)
total_characters = string.ascii_lowercase+' .'
no_of_characters = len(total_characters)

In [7]:
total_characters, seq_len

('abcdefghijklmnopqrstuvwxyz .', 106)

In [8]:
# create one-hot encoding
def one_hot_encoding(char):
    encode = torch.zeros(no_of_characters)
    encode[total_characters.find(char)]=1
    return encode
    
def getLine(seq):
    encode_list = []
    for ch in seq:
        encode_list.append(one_hot_encoding(ch))
    return torch.cat(encode_list,dim=0).view(len(seq),1,no_of_characters)

In [9]:
class MyLSTM(nn.Module):
    def __init__(self,input_dim,hidden_dim):
        super(MyLSTM,self).__init__()
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.LSTM = nn.LSTM(input_dim,hidden_dim)
        self.LNN = nn.Linear(hidden_dim,input_dim)
        
    def forward(self,inp,hc):
        output,_= self.LSTM(inp,hc)
        return self.LNN(output)

In [10]:
# build model
hidden_dim = no_of_characters     
model = MyLSTM(no_of_characters,hidden_dim)
# optimizer function
optimizer = torch.optim.Adam(params = model.parameters(),lr=0.01)
#cross_entropy_loss
LOSS = torch.nn.CrossEntropyLoss()

In [11]:
targets = []
#Iterate through all characters in a sequence. We pass first character as input, so starts from 2nd position 
for x in input_[1:]+'.':
    targets.append(total_characters.find(x))

In [12]:
#Convert into tensor
targets = torch.tensor(targets)
n_iters = 400
inpl = []
#Iterate through all inputs in the sequence
for ch in input_:
    #Convert into tensor
    inpl.append(one_hot_encoding(ch))

#Convert list to tensor
inp = torch.cat(inpl,dim=0)
inp = inp.view(seq_len,1,no_of_characters)

In [16]:
import time
start = time.time()
n_iters = 150
for itr in range(n_iters):
    model.zero_grad()
    optimizer.zero_grad()
    #Initialize h and c vectors
    h = torch.rand(hidden_dim).view(1,1,hidden_dim)
    c = torch.rand(hidden_dim).view(1,1,hidden_dim)
    output = model(inp,(h,c))
    output = output.view(seq_len,no_of_characters)
    #Find loss
    loss = LOSS(output,targets)
    if itr%10==0:
        print('Iteration : '+str(itr)+' Loss : '+str(loss) )
    #Back propagate the loss
    loss.backward()
    optimizer.step()

Iteration : 0 Loss : tensor(0.1036, grad_fn=<NllLossBackward>)
Iteration : 10 Loss : tensor(0.0822, grad_fn=<NllLossBackward>)
Iteration : 20 Loss : tensor(0.0686, grad_fn=<NllLossBackward>)
Iteration : 30 Loss : tensor(0.0578, grad_fn=<NllLossBackward>)
Iteration : 40 Loss : tensor(0.0494, grad_fn=<NllLossBackward>)
Iteration : 50 Loss : tensor(0.0428, grad_fn=<NllLossBackward>)
Iteration : 60 Loss : tensor(0.0378, grad_fn=<NllLossBackward>)
Iteration : 70 Loss : tensor(0.0336, grad_fn=<NllLossBackward>)
Iteration : 80 Loss : tensor(0.0302, grad_fn=<NllLossBackward>)
Iteration : 90 Loss : tensor(0.0273, grad_fn=<NllLossBackward>)
Iteration : 100 Loss : tensor(0.0248, grad_fn=<NllLossBackward>)
Iteration : 110 Loss : tensor(0.0227, grad_fn=<NllLossBackward>)
Iteration : 120 Loss : tensor(0.0208, grad_fn=<NllLossBackward>)
Iteration : 130 Loss : tensor(0.0193, grad_fn=<NllLossBackward>)
Iteration : 140 Loss : tensor(0.0177, grad_fn=<NllLossBackward>)


In [18]:
 def predict(s):
    """
    given input sequnce, it will predict new character
    """
    inp = getLine(s)
    #Initialize h and c vectors
    h = torch.rand(1,1,hidden_dim)
    c = torch.rand(1,1,hidden_dim)
    #Get the output
    out = model(inp,(h,c))
    #Find the corresponding letter from the output
    return total_characters[out[-1][0].topk(1)[1].detach().numpy().item()]

In [19]:
predict('brand m')

'a'

In [17]:
def continuous(ch):
    if ch[-1]=='.' or len(ch)>=len(input_)+5:
        print(ch)
        return
    pred = predict(ch)
    print(ch+pred)
    #recursive
    continuous(ch+pred)

In [ ]:
continuous('b')

br
bra
bran
brand
brand 
brand m
brand ma
brand map
brand mapp
brand mappi
brand mappin
brand mapping
brand mapping 
brand mapping i
brand mapping is
brand mapping is 
brand mapping is u
brand mapping is us
brand mapping is use
brand mapping is used
brand mapping is used 
brand mapping is used t
brand mapping is used to
brand mapping is used to 
brand mapping is used to u
brand mapping is used to un
brand mapping is used to und
brand mapping is used to unde
brand mapping is used to under
brand mapping is used to unders
brand mapping is used to underst
brand mapping is used to understa
brand mapping is used to understan
brand mapping is used to understand
brand mapping is used to understand 
brand mapping is used to understand t
brand mapping is used to understand th
brand mapping is used to understand the
brand mapping is used to understand the 
brand mapping is used to understand the p
brand mapping is used to understand the po
brand mapping is used to understand the pos
brand mapping

brand mapping is used to understand the positioning of a brand with respect to the attributes in any brand with respect to the at
brand mapping is used to understand the positioning of a brand with respect to the attributes in any brand with respect to the att
brand mapping is used to understand the positioning of a brand with respect to the attributes in any brand with respect to the attr
brand mapping is used to understand the positioning of a brand with respect to the attributes in any brand with respect to the attri
brand mapping is used to understand the positioning of a brand with respect to the attributes in any brand with respect to the attrib
brand mapping is used to understand the positioning of a brand with respect to the attributes in any brand with respect to the attribu
brand mapping is used to understand the positioning of a brand with respect to the attributes in any brand with respect to the attribut
brand mapping is used to understand the positioning of a brand with r

brand mapping is used to understand the positioning of a brand with respect to the attributes in any brand with respect to the attributes in any brand with respect to the attributes in a
brand mapping is used to understand the positioning of a brand with respect to the attributes in any brand with respect to the attributes in any brand with respect to the attributes in an
brand mapping is used to understand the positioning of a brand with respect to the attributes in any brand with respect to the attributes in any brand with respect to the attributes in any
brand mapping is used to understand the positioning of a brand with respect to the attributes in any brand with respect to the attributes in any brand with respect to the attributes in any 
brand mapping is used to understand the positioning of a brand with respect to the attributes in any brand with respect to the attributes in any brand with respect to the attributes in any b
brand mapping is used to understand the positioning of 

brand mapping is used to understand the positioning of a brand with respect to the attributes in any brand with respect to the attributes in any brand with respect to the attributes in any brand with respect to the attributes i
brand mapping is used to understand the positioning of a brand with respect to the attributes in any brand with respect to the attributes in any brand with respect to the attributes in any brand with respect to the attributes in
brand mapping is used to understand the positioning of a brand with respect to the attributes in any brand with respect to the attributes in any brand with respect to the attributes in any brand with respect to the attributes in 
brand mapping is used to understand the positioning of a brand with respect to the attributes in any brand with respect to the attributes in any brand with respect to the attributes in any brand with respect to the attributes in a
brand mapping is used to understand the positioning of a brand with respect to the

brand mapping is used to understand the positioning of a brand with respect to the attributes in any brand with respect to the attributes in any brand with respect to the attributes in any brand with respect to the attributes in any brand with respect to the attr
brand mapping is used to understand the positioning of a brand with respect to the attributes in any brand with respect to the attributes in any brand with respect to the attributes in any brand with respect to the attributes in any brand with respect to the attri
brand mapping is used to understand the positioning of a brand with respect to the attributes in any brand with respect to the attributes in any brand with respect to the attributes in any brand with respect to the attributes in any brand with respect to the attrib
brand mapping is used to understand the positioning of a brand with respect to the attributes in any brand with respect to the attributes in any brand with respect to the attributes in any brand with respe

brand mapping is used to understand the positioning of a brand with respect to the attributes in any brand with respect to the attributes in any brand with respect to the attributes in any brand with respect to the attributes in any brand with respect to the attributes in any brand with respec
brand mapping is used to understand the positioning of a brand with respect to the attributes in any brand with respect to the attributes in any brand with respect to the attributes in any brand with respect to the attributes in any brand with respect to the attributes in any brand with respect
brand mapping is used to understand the positioning of a brand with respect to the attributes in any brand with respect to the attributes in any brand with respect to the attributes in any brand with respect to the attributes in any brand with respect to the attributes in any brand with respect 
brand mapping is used to understand the positioning of a brand with respect to the attributes in any brand with 

brand mapping is used to understand the positioning of a brand with respect to the attributes in any brand with respect to the attributes in any brand with respect to the attributes in any brand with respect to the attributes in any brand with respect to the attributes in any brand with respect to the attributes in any b
brand mapping is used to understand the positioning of a brand with respect to the attributes in any brand with respect to the attributes in any brand with respect to the attributes in any brand with respect to the attributes in any brand with respect to the attributes in any brand with respect to the attributes in any br
brand mapping is used to understand the positioning of a brand with respect to the attributes in any brand with respect to the attributes in any brand with respect to the attributes in any brand with respect to the attributes in any brand with respect to the attributes in any brand with respect to the attributes in any bra
brand mapping is used to und

brand mapping is used to understand the positioning of a brand with respect to the attributes in any brand with respect to the attributes in any brand with respect to the attributes in any brand with respect to the attributes in any brand with respect to the attributes in any brand with respect to the attributes in any brand with respect to the at
brand mapping is used to understand the positioning of a brand with respect to the attributes in any brand with respect to the attributes in any brand with respect to the attributes in any brand with respect to the attributes in any brand with respect to the attributes in any brand with respect to the attributes in any brand with respect to the att
brand mapping is used to understand the positioning of a brand with respect to the attributes in any brand with respect to the attributes in any brand with respect to the attributes in any brand with respect to the attributes in any brand with respect to the attributes in any brand with respect to 